In [1]:
!git clone https://github.com/llm-fine-tuning/LLaMA-Factory.git

fatal: destination path 'LLaMA-Factory' already exists and is not an empty directory.


In [2]:
%cd LLaMA-Factory

/workspace/LLaMA-Factory


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
!pip install -r requirements.txt


[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip


In [4]:
!pip install deepspeed==0.14


[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip


In [10]:
# 학습 스크립트 확인
file_path = 'train_sft.sh'

with open(file_path, 'r') as file:
    content = file.read()
    print(content)

deepspeed --num_gpus 2 --master_port=9901 src/train.py \
	--deepspeed ds_z3_config.json \
    --stage sft \
    --do_train \
    --model_name_or_path NCSOFT/Llama-VARCO-8B-Instruct \
    --dataset hate_speech_detection \
    --template llama3-hate-speech-detection \
    --finetuning_type lora \
    --lora_target all \
    --output_dir checkpoint \
    --overwrite_cache \
    --per_device_train_batch_size 4 \
    --gradient_accumulation_steps 4 \
    --lr_scheduler_type cosine \
    --logging_steps 10 \
    --save_steps 100 \
    --learning_rate 1e-4 \
    --num_train_epochs 10.0 \
    --report_to none \
    --bf16

    


## 학습 진행 후 체크포인트가 생기면 merge

In [11]:
!python merge.py \
    --base_model_name_or_path NCSOFT/Llama-VARCO-8B-Instruct \
    --peft_model_path ./checkpoint/checkpoint-100 \
    --output_dir ./output_dir

Loading base model: NCSOFT/Llama-VARCO-8B-Instruct
Loading checkpoint shards: 100%|██████████████████| 4/4 [00:17<00:00,  4.37s/it]
Loading PEFT: ./checkpoint/checkpoint-100
Running merge_and_unload
[2024-10-18 10:10:35,329] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
Model saved to ./output_dir


## 학습 전 기본 모델 호출


In [12]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [15]:
tokenizer = AutoTokenizer.from_pretrained('NCSOFT/Llama-VARCO-8B-Instruct')

In [16]:
tokenizer.convert_tokens_to_ids("<|eot_id|>")

128009

In [17]:
tokenizer = AutoTokenizer.from_pretrained('NCSOFT/Llama-VARCO-8B-Instruct')
model = AutoModelForCausalLM.from_pretrained('NCSOFT/Llama-VARCO-8B-Instruct')
model = torch.nn.DataParallel(model).cuda()

input_text = '''<|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 주어진 입력이 어떤 혐오 카테고리에 속하는지를 분류하는 분류기입니다.

1. 아래의 카테고리 중에서만 고르세요.
['출신차별', '외모차별', '정치성향차별', '혐오욕설', '연령차별', '성차별', '인종차별', '종교차별', '혐오아님']
2. 주어진 입력은 위 8개의 카테고리 중 여러 개에 속할 수 있습니다.
3. 출력은 반드시 파이썬의 리스트 형태로 생성하십시오.
4. 입력:이 주어지면 분류 결과: 다음에 파이썬 리스트 형태로 작성하세요.<|eot_id|><|start_header_id|>user<|end_header_id|>

입력: "자한당틀딱들.. 악플질 고만해라."<|eot_id|><|start_header_id|>assistant<|end_header_id|>분류 결과:'''

inputs = tokenizer(input_text, return_tensors="pt")
eos_token_id = tokenizer.convert_tokens_to_ids("<|eot_id|>")

with torch.no_grad():
    outputs = model.module.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=512, eos_token_id=eos_token_id)
    print(tokenizer.decode(outputs[0]))

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 주어진 입력이 어떤 혐오 카테고리에 속하는지를 분류하는 분류기입니다.

1. 아래의 카테고리 중에서만 고르세요.
['출신차별', '외모차별', '정치성향차별', '혐오욕설', '연령차별', '성차별', '인종차별', '종교차별', '혐오아님']
2. 주어진 입력은 위 8개의 카테고리 중 여러 개에 속할 수 있습니다.
3. 출력은 반드시 파이썬의 리스트 형태로 생성하십시오.
4. 입력:이 주어지면 분류 결과: 다음에 파이썬 리스트 형태로 작성하세요.<|eot_id|><|start_header_id|>user<|end_header_id|>

입력: "자한당틀딱들.. 악플질 고만해라."<|eot_id|><|start_header_id|>assistant<|end_header_id|>분류 결과: ['혐오욕설', '성차별']

설명:
주어진 문장에서 "자한당틀딱들"이라는 표현은 상대방을 모욕하고 비하하는 내용을 담고 있습니다. 이는 '혐오욕설' 카테고리에 속합니다. 또한 "악플질 고만해라"라는 표현은 성차별적인 태도를 드러내고 있습니다. 여성이 '악플'이라는 단어를 사용하면서도 상대방을 제어하려고 하는 성차별적 시각을 보여주고 있기 때문입니다. 따라서 이 문장은 '성차별' 카테고리에도 속한다고 볼 수 있습니다. 

Python 리스트 형태로 표현하면 다음과 같습니다:
['혐오욕설', '성차별']<|eot_id|>


## 학습 후 모델 호출

In [19]:
tokenizer = AutoTokenizer.from_pretrained('./output_dir')
model = AutoModelForCausalLM.from_pretrained('./output_dir')
model = torch.nn.DataParallel(model).cuda()

input_text = '''<|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 주어진 입력이 어떤 혐오 카테고리에 속하는지를 분류하는 분류기입니다.

1. 아래의 카테고리 중에서만 고르세요.
['출신차별', '외모차별', '정치성향차별', '혐오욕설', '연령차별', '성차별', '인종차별', '종교차별', '혐오아님']
2. 주어진 입력은 위 8개의 카테고리 중 여러 개에 속할 수 있습니다.
3. 출력은 반드시 파이썬의 리스트 형태로 생성하십시오.
4. 입력:이 주어지면 분류 결과: 다음에 파이썬 리스트 형태로 작성하세요.<|eot_id|><|start_header_id|>user<|end_header_id|>

입력: "자한당틀딱들.. 악플질 고만해라."<|eot_id|><|start_header_id|>assistant<|end_header_id|>분류 결과:'''

inputs = tokenizer(input_text, return_tensors="pt")
eos_token_id = tokenizer.convert_tokens_to_ids("<|eot_id|>")

with torch.no_grad():
    outputs = model.module.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=512, eos_token_id=eos_token_id)
    print(tokenizer.decode(outputs[0]))

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 주어진 입력이 어떤 혐오 카테고리에 속하는지를 분류하는 분류기입니다.

1. 아래의 카테고리 중에서만 고르세요.
['출신차별', '외모차별', '정치성향차별', '혐오욕설', '연령차별', '성차별', '인종차별', '종교차별', '혐오아님']
2. 주어진 입력은 위 8개의 카테고리 중 여러 개에 속할 수 있습니다.
3. 출력은 반드시 파이썬의 리스트 형태로 생성하십시오.
4. 입력:이 주어지면 분류 결과: 다음에 파이썬 리스트 형태로 작성하세요.<|eot_id|><|start_header_id|>user<|end_header_id|>

입력: "자한당틀딱들.. 악플질 고만해라."<|eot_id|><|start_header_id|>assistant<|end_header_id|>분류 결과: ['연령차별']<|eot_id|>
